# make dataframe

#### I got the image information by [[siim-step1]get_imginfo](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

#### Next step, I will make the dataframe based on image information.

## my public notebook 

### [step1 get_imageinformation](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

### [step2 make_dataframe](https://www.kaggle.com/kunihikofurugori/siim-step1-get-imginfo).

### [step3-1 renew-imglev_ds](https://www.kaggle.com/kunihikofurugori/siim-step3-1-renew-imglev-ds)

### [step3-2 renew-studylev_ds](https://www.kaggle.com/kunihikofurugori/siim-step3-1-renew-studylev-ds)

In [ ]:
!pip install python-gdcm
import numpy as np
import pandas as pd
import cv2,random
import pydicom
import gdcm
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image, ImageStat

In [ ]:
path = glob.glob("../input/siim-covid19-detection/train/**/*.dcm", recursive=True)
df_img = pd.read_csv("../input/siim-covid19-detection/train_image_level.csv")
df_label = pd.read_csv("../input/siim-covid19-detection/train_study_level.csv")
df_img.StudyInstanceUID = df_img.StudyInstanceUID + "_study"
df = pd.merge(df_img, df_label, left_on='StudyInstanceUID', right_on='id').drop(columns='id_y')
path_df = pd.DataFrame(path, columns=['path'])
path_df['filename'] = path_df.path.apply(lambda p: os.path.split(p)[1])
path_df.filename = path_df.filename.str.replace('.dcm', '_image')
df = pd.merge(df, path_df, left_on='id_x', right_on='filename').drop(columns='filename')
df['fold'] = 0
df.fold = df.fold.apply(lambda x: random.randint(0,3))
df.head()

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

def resize(array, size1,size2, keep_ratio=True, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size1, size2), resample)
        im = expand2square(im, background_color=0)
    else:
        im = im.resize((size1, size2), resample)
    
    return im

In [ ]:
def kuroobi_hrange(array, black_thr=50, white_thr=250):
    H, W = np.shape(array)
    cut_dh0, cut_dh1 = 0,0
    for h0 in range(0, H//2):
        row = array[h0, :]
        mean = np.sum(row)/ len(row)
        if mean < black_thr or white_thr < mean:
            cut_dh0 += 1
        else:
            break
    for h1 in range(1, H//2):
        row = array[H-h1, :]
        mean = np.sum(row) / len(row)
        if mean < black_thr or white_thr < mean:
            cut_dh1 += 1
        else:
            break
    return cut_dh0, cut_dh1

def kuroobi_wrange(array, black_thr=50, white_thr=250):
    H, W = np.shape(array)
    cut_dw0, cut_dw1 = 0,0
    for w0 in range(0, W//2):
        col = array[:,w0]
        mean = np.sum(col) / len(col)
        if mean < black_thr or white_thr < mean:
            cut_dw0 += 1
        else:
            break
    for w1 in range(1, W//2):
        col = array[:,W-w1]
        mean = np.sum(col) / len(col)
        if mean < black_thr or white_thr < mean:
            cut_dw1 += 1
        else:
            break
    return cut_dw0, cut_dw1

def check_kuroobi(image, black_thr=50, white_thr=250):
    array = np.asarray(image)
    H, W = np.shape(array)
    dh0, dh1 = kuroobi_hrange(array)
    dw0, dw1 = kuroobi_wrange(array)
    return dh0,dw0,H-dh1,W-dw1


In [ ]:
dfs = []
dfs.append(df[df.fold ==0])
dfs.append(df[df.fold ==1])
dfs.append(df[df.fold ==2])
dfs.append(df[df.fold ==3])
print(len(dfs[0]),len(dfs[1]),len(dfs[2]),len(dfs[3]))
print(len(df))

In [ ]:
df['mean'] = 0
df['var'] = 0
df['dim1'] = 0
df['dim2'] = 0

df['crop_xmin'] = 0
df['crop_ymin'] = 0
df['crop_xmax'] = 0
df['crop_ymax'] = 0

df['crop_fxmin'] = 0
df['crop_fymin'] = 0
df['crop_fxmax'] = 0
df['crop_fymax'] = 0

In [ ]:
def run(fold):
    for idx,(index,row) in enumerate(dfs[fold].iterrows()):
        image = read_xray(row.path)
        dim0, dim1 = image.shape
        df.loc[index,'dim1'], df.loc[index,'dim2'] = dim0, dim1
        
        xmin, ymin, xmax, ymax = check_kuroobi(image)
        
        df.loc[index,"crop_xmin"] = xmin
        df.loc[index,"crop_ymin"] = ymin
        df.loc[index,"crop_xmax"] = xmax
        df.loc[index,"crop_ymax"] = ymax
        
        df.loc[index,"crop_fxmin"] = xmin/dim0
        df.loc[index,"crop_fymin"] = ymin/dim1
        df.loc[index,"crop_fxmax"] = xmax/dim0
        df.loc[index,"crop_fymax"] = ymax/dim1

        df.loc[index,'mean'] = image.mean()
        df.loc[index,'var'] = image.std()
        
        if idx%500==0:
            print(idx)

In [ ]:
from joblib import Parallel, delayed
Parallel(n_jobs=4, backend="threading")(delayed(run)(i) for i in range(4))

In [ ]:
df.to_csv("imginfo.csv")